In [118]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import re
import csv

In [119]:
data = pd.DataFrame(columns=['name', 'image', 'description', 'location', 'speaking', 'rating', 'num_reviews' ,
                                     'currency', 'salary', 'num_proj', 'num_buyers', 'last_active', 'expertise',
                                      'skills', 'gender'])

In [120]:
profiles = []

with open('profiles_url.csv', 'r', encoding="utf8") as csvfile:
    csvreader = csv.reader(csvfile)
    fields = next(csvreader)
  
    for row in csvreader:
        profiles.append(row[1])

In [121]:
URL_list = ['https://www.peopleperhour.com/freelancer/marketing-seo/gajura-constantin-seo-expert-and-experienced-guest-wwanwy',
            'https://www.peopleperhour.com/freelancer/writing-translation/logical_translation_localisation-translations-translation-localisation-mnyay',
            'https://www.peopleperhour.com/freelancer/writing-translation/karen-siddall-cv-linkedin-writer-for-the-legal-vjjjyvm',
            'https://www.peopleperhour.com/freelancer/writing-translation/michelle-vanwettere-1-professional-translator-on-pph-yznqxqy',
            'https://www.peopleperhour.com/freelancer/writing-translation/artisticpen-translation-voice-over-social-ywyjqw']

In [122]:
def guess_gender(rev):
    """
    determine sex of a worker with a list of reviews
    """
    gender = ""
    male = ["he", "his", "him", "himself"]
    female = ["she", "her", "hers", "herself"]
    
    for r in rev :
        for word in male :
            if re.compile(r'\b({0})\b'.format(word), flags=re.IGNORECASE).search(r) != None:
                gender = "male"
                return gender
        for word in female : 
            if re.compile(r'\b({0})\b'.format(word), flags=re.IGNORECASE).search(r) != None:
                gender = "female"    
                return gender
    return gender

In [123]:
data = pd.DataFrame(columns=['name', 'image', 'description', 'location', 'speaking', 'rating', 'num_reviews' ,
                         'currency', 'salary', 'num_proj', 'num_buyers', 'last_active', 'expertise',
                         'skills', 'gender'])

for URL in profiles[0:20]:

    r = requests.get(URL)
    soup = BeautifulSoup(r.content)

    location = soup.find("div", {"class": "member-location"}).get_text().replace('\n', '').replace('\r', '') if soup.find("div", {"class": "member-location"}) else ""
    speaking = soup.find("div", {"class": "member-languages clearfix"}).get_text().replace('\n', '').replace('\r', '') if soup.find("div", {"class": "member-languages clearfix"}) else ""
    rating = soup.find("div", {"class": "total-rating"}).get_text() if soup.find("div", {"class": "total-rating"}) else ""
    num_reviews = int(soup.find("div", {"class": "total-reviews"}).get_text().replace('(', '').replace(')', '')) if soup.find("div", {"class": "total-reviews"}) else ""
    cost = soup.find("span", {"class": "member-cost"}).get_text().replace('\n', '').replace('\r', '') if soup.find("span", {"class": "member-cost"}) else ""
    currency = cost[0] if cost else ""
    salary = int(cost[1:].replace('/hr', '')) if cost else ""
    num_proj = int(re.sub("[^0-9]", "", soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ['memStats-container']).find("div", {"class": "memberStats-item memberStats-rating"}).get_text().replace('\r', '').replace('\n', ''))) if soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ['memStats-container']).find("div", {"class": "memberStats-item memberStats-rating"}) else ""
    num_buyers = int(re.sub("[^0-9]", "", soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ['memStats-container']).find("div", {"class": "memberStats-item u-mgb--1"}).get_text().replace('\r', '').replace('\n', ''))) if soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ['memStats-container']).find("div", {"class": "memberStats-item u-mgb--1"}) else ""
    last_active = soup.find("div", {"class": "memberStats-item item-margin"}).find("div", {"class": "insights-value"}).get_text().replace('\n', '').replace('\r', '') if soup.find("div", {"class": "memberStats-item item-margin"}).find("div", {"class": "insights-value"}) else ""
    expertise = soup.find("div", {"class": "clearfix industry-expertise-list skills-item"}).get_text().replace('\n', '').replace('\r', '') if soup.find("div", {"class": "clearfix industry-expertise-list skills-item"}) else ""
    response_time = soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ['memberStats-item']).find("div", {"class": "insights-value"}).get_text().replace('\n', '').replace('\r', '') if soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ['memberStats-item']).find("div", {"class": "insights-value"}) else ""
    skills = soup.find("div", {"class": "col-md-12 col-lg-12"}).get_text().replace('\n', '').replace('\r', '') if soup.find("div", {"class": "col-md-12 col-lg-12"}) else ""

    profile = json.loads("".join(soup.find("script", {"type":"application/ld+json"}).contents)) if soup.find("script", {"type":"application/ld+json"}) else ""
    name = profile['name'].replace('\n', '').replace('\r', '') if profile['name'] else ""
    image = profile['image'].replace('\n', '').replace('\r', '') if profile['image'] else ""
    description = profile['description'].replace('\n', '').replace('\r', '').replace('\t', '') if profile['description'] else ""

    reviews = [com.find("p").get_text() for com in soup.findAll("li", {"class": "item participant feedback clearfix"})] if soup.findAll("li", {"class": "item participant feedback clearfix"}) else ""
    gender = guess_gender(reviews) if reviews else ""   
    
    worker = [name, image, description, location, speaking, rating, num_reviews,currency, salary, num_proj,
              num_buyers, last_active, expertise,skills, gender]

    series = pd.Series(worker, index = data.columns)
    data = data.append(series, ignore_index=True)

In [124]:
data

,name,image,description,location,speaking,rating,num_reviews,currency,salary,num_proj,num_buyers,last_active,expertise,skills,gender
0,Hannah Hutchins,https://dw3i9sxi97owk.cloudfront.net/uploads/6...,I am an audio transcriptionist with over four ...,"Coventry, United Kingdom",English speaking,5.0,24,£,10,24,17,Apr 21 2021,"Industry expertise: Healthcare, Legal ...",Skillstranscriptioncopy typingaudio/sound tran...,female
1,Translate Guru .,https://dw3i9sxi97owk.cloudfront.net/uploads/b...,Hello! Hola! Ciao! Bonjour! 你好!I do high-quali...,"Cannanore, India",,4.9,597,$,20,719,375,Apr 21 2021,,SkillsGerman <=> English translationArabic <=>...,
2,Logical Translation & Localisation,https://dw3i9sxi97owk.cloudfront.net/uploads/3...,OFFERING A WIDE RANGE OF TRANSLATION SERVICESA...,"City of London, United Kingdom","English, Spanish, French and Italian speaking",4.9,135,£,20,140,100,Apr 21 2021,"Industry expertise: Advertising, Aeros...",SkillseditingproofreadingGerman translationEng...,female
3,Al Nas,https://dw3i9sxi97owk.cloudfront.net/uploads/0...,"QUALITY, SPEED and COMMUNICATION: These are th...","Ystrad Mynach, United Kingdom","German, English, Spanish, French, Italian, Dut...",4.9,552,£,16,401,292,Apr 21 2021,"Industry expertise: Advertising, Educa...",SkillstranscriptionGerman <=> English translat...,
4,Karen Siddall,https://dw3i9sxi97owk.cloudfront.net/uploads/4...,As an ex lawyer and experienced legal recruite...,"Warrington, United Kingdom",English speaking,,,£,75,1,1,Oct 25 2020,"Industry expertise: Human Resources, L...",Skillslinkedin page designbusiness development...,
5,Michelle Vanwettere,https://dw3i9sxi97owk.cloudfront.net/uploads/9...,"I am a native Dutch, German and English speake...","Hoogstraten, Belgium","German, English, French and Dutch speaking",5.0,198,€,40,133,91,Apr 21 2021,"Industry expertise: Advertising, Legal...",Skillssocial media managementGerman <=> Englis...,female
6,ArtisticPen,https://dw3i9sxi97owk.cloudfront.net/uploads/c...,"We believe that Translation & Writing is an ""A...","Mumbai, India","German, Spanish, French, Italian, Dutch and Ru...",4.9,92,£,25,70,55,Apr 19 2021,,SkillsGerman <=> English translationGerman tra...,
7,Dina Lourenço,https://dw3i9sxi97owk.cloudfront.net/uploads/2...,"Hi,I'm a Portuguese (native speaker) freelance...","Serta, Portugal",English speaking,5.0,273,€,15,205,136,Apr 21 2021,,Skillseditingmicrosoft exceloffice managementp...,female
8,Donene Diedericks,https://dw3i9sxi97owk.cloudfront.net/uploads/1...,"I am a native English professional, who is als...","Pretoria, South Africa",Afrikaans and English speaking,5.0,27,$,25,26,18,Apr 21 2021,"Industry expertise: Education, Media a...",Skillseditingproofreadinginternet researcharti...,female
9,Fast Translations and Multilanguage Solutions ...,https://dw3i9sxi97owk.cloudfront.net/uploads/6...,Quality Translation:FTMLS is a leading provide...,"Gwalior, India",English speaking,4.7,111,£,15,171,94,Apr 21 2021,"Industry expertise: Advertising, Aeros...",SkillsFrench <=> English translationNorwegian ...,


In [125]:
data.to_csv('./database.csv', index=False)